## Problem Statement
Dalam pengerjaan koreksi satelit, perlu dilakukan windowing berdasarkan kategori hujan, sehingga dibutuhkan tahapan untuk mengubah data hujan berdasarkan kategori hujan dari BMKG. Adapun dalam pengerjaan koreksi data satelit, dibutuhkan data hujan observasi yang berperan sebagai __corrector__ dan adapula data satelit yang ingin dikoreksi di daerah irigasi. Biasanya data satelit digunakan untuk area yang tidak memiliki data ch obs sehingga akan dilakukan koreksi CH satelit menggunakan data CH dari stasiun observasi di lokasi terdekat.

## Objective
- Script ini digunakan untuk mengklasifikasikan data hujan ke dalam kategori ringan, sedang, lebat, dan sangat lebat.

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import glob
from datetime import datetime, timedelta

In [2]:
# read gpm data
obs = pd.read_excel("D:/Happy/RTOW/Pemali Juana/koreksi gpm/CH koreksi/Dipakai/utk koreksi fix/CH_obsQC.xlsx")
obs=obs.iloc[:,1:]
obs['date'] = pd.to_datetime(obs['date'], errors='coerce')
path_gpm =r'D:\Happy\RTOW\Pemali Juana\koreksi gpm\gpm result model\model'
idxs_gpm = glob.glob(path_gpm + "/*.xlsx")
model_gpm=idxs_gpm[0][81:-5]

In [3]:
file_gpm = glob.glob(path_gpm + "/*.xlsx")[0]
print (idxs_gpm[0][81:-5])
init=pd.read_excel(idxs_gpm[0])
init

M11_Lusi


,date,S2,S5,S6
0,01-06-2000,0.012913,0.041005,0.335348
1,02-06-2000,14.873110,11.260475,12.208823
2,03-06-2000,3.028746,3.690391,7.836700
3,04-06-2000,3.699177,0.372721,0.075601
4,05-06-2000,10.145528,8.638637,0.445616
...,...,...,...,...
7910,27-01-2022,4.351445,8.308703,3.828028
7911,28-01-2022,1.936565,2.676108,1.406599
7912,29-01-2022,1.326366,0.890306,0.620490
7913,30-01-2022,4.041034,5.975636,1.785773


In [4]:
init.empty

False

In [8]:
idxs_gpm

['D:\\Happy\\RTOW\\Pemali Juana\\koreksi gpm\\gpm result model\\kebutuhan\\result_Gage_Weight_FaktorAreaKlambu.xlsx',
 'D:\\Happy\\RTOW\\Pemali Juana\\koreksi gpm\\gpm result model\\kebutuhan\\result_Gage_Weight_FaktorAreaSedadi.xlsx',
 'D:\\Happy\\RTOW\\Pemali Juana\\koreksi gpm\\gpm result model\\kebutuhan\\result_Gage_Weight_FaktorAreaSidorejo.xlsx']

In [9]:
init.date

0       01-06-2000
1       02-06-2000
2       03-06-2000
3       04-06-2000
4       05-06-2000
           ...    
7910    27-01-2022
7911    28-01-2022
7912    29-01-2022
7913    30-01-2022
7914    31-01-2022
Name: date, Length: 7915, dtype: object

In [10]:
init['date'] = pd.to_datetime(init['date'], format="%d-%m-%Y")
init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7915 entries, 0 to 7914
Data columns (total 33 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    7915 non-null   datetime64[ns]
 1   S1      7915 non-null   float64       
 2   S3      7915 non-null   float64       
 3   S4      7915 non-null   float64       
 4   S5      7915 non-null   float64       
 5   S7      7915 non-null   float64       
 6   S8      7915 non-null   float64       
 7   S9      7915 non-null   float64       
 8   S10     7915 non-null   float64       
 9   S11     7915 non-null   float64       
 10  S12     7915 non-null   float64       
 11  S14     7915 non-null   float64       
 12  S15     7915 non-null   float64       
 13  S16     7915 non-null   float64       
 14  S17     7915 non-null   float64       
 15  S18     7915 non-null   float64       
 16  S19     7915 non-null   float64       
 17  S20     7915 non-null   float64       
 18  S21     

In [11]:
with pd.ExcelWriter('D:/Happy/RTOW/Pemali Juana/koreksi gpm/test gabung raw/gpm_model.xlsx') as writer:
    init.date.to_excel(writer)

In [12]:
idxs_gpm[-1][95:-5]

'Sidorejo'

In [13]:
#loop read excel gpm per model
path_gpm =r'D:\Happy\RTOW\Pemali Juana\koreksi gpm\gpm result model\model'
idxs_gpm = glob.glob(path_gpm + "/*.xlsx")

for file in idxs_gpm:
    #get gpm data
    model_gpm=file[81:-5]
    file_gpm = glob.glob(path_gpm + "/*"+model_gpm+".xlsx")[0]
    globals()["gpm_"+model_gpm]=pd.read_excel(file_gpm)
    globals()["gpm_"+model_gpm]['date'] = pd.to_datetime(globals()["gpm_"+model_gpm]['date'], format="%d-%m-%Y")
    name_tmp=globals()["gpm_"+model_gpm].columns[1:].values
    
    #Klasifikasi hujan
    # loop berdasarkan urutan di excel stasiun gpm per gage tiap model
    for i in range (len(name_tmp)):
        gage=name_tmp[i]

        #create dataframe with obs data
        gpm_tmp={"date":globals()["gpm_"+model_gpm].loc[:,"date"],gage:globals()["gpm_"+model_gpm].loc[:,gage]}
        globals()[model_gpm+"_gage"+str(i)]=pd.DataFrame(gpm_tmp)

        #klasifikasi obs
        conditions = [
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] >= 0.) &\
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage]< 20),
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] >= 20) & \
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] < 50),
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] >= 50) & \
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] < 100),
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] >= 100)
            ]

        # create a list of the values we want to assign for each condition
        values = ['Ringan','Sedang','Lebat','Sangat_Lebat']

        # create a new column and use np.select to assign values to it using our lists as arguments
        globals()[model_gpm+"_gage"+str(i)]['Klasifikasi'] = np.select(conditions, values)

        print ('Done:  '+model_gpm+' '+gage)

        #Split gpm table based on rainfall classification for each ch area
    for k in range (len(name_tmp)) :
        for j in range (len(values)) :
            globals()[values[j]+model_gpm+"_gage"+str(k)] = \
            globals()[model_gpm+"_gage"+str(k)][globals()[model_gpm+"_gage"+str(k)]['Klasifikasi']==values[j]].reset_index(drop=True)
            #print (values[j]+model_gpm+"_gage"+str(k))
                
        
        #save into excel per model dan klasifikasi
    for n in values:
        with pd.ExcelWriter('D:/Happy/RTOW/Pemali Juana/koreksi gpm/klasifikasi ch/model/qc_'\
                            +n+'_'+model_gpm+'.xlsx') as writer: 
            for o in range (len(name_tmp)):
                gage=name_tmp[o]
                globals()[n+model_gpm+"_gage"+str(o)].to_excel(writer, sheet_name=gage)
    
    

Done:  Klambu S1
Done:  Klambu S3
Done:  Klambu S4
Done:  Klambu S5
Done:  Klambu S7
Done:  Klambu S8
Done:  Klambu S9
Done:  Klambu S10
Done:  Klambu S11
Done:  Klambu S12
Done:  Klambu S14
Done:  Klambu S15
Done:  Klambu S16
Done:  Klambu S17
Done:  Klambu S18
Done:  Klambu S19
Done:  Klambu S20
Done:  Klambu S21
Done:  Klambu S23
Done:  Klambu S24
Done:  Klambu S25
Done:  Klambu S26
Done:  Klambu S27
Done:  Klambu S30
Done:  Klambu S31
Done:  Klambu S34
Done:  Klambu S35
Done:  Klambu S36
Done:  Klambu S37
Done:  Klambu S39
Done:  Klambu S42
Done:  Klambu S43
Done:  Sedadi s6
Done:  Sedadi s7
Done:  Sedadi s8
Done:  Sedadi s12
Done:  Sedadi s13
Done:  Sedadi s14
Done:  Sedadi s15
Done:  Sedadi s19
Done:  Sedadi s20
Done:  Sedadi s21
Done:  Sedadi s22
Done:  Sedadi s25
Done:  Sedadi s26
Done:  Sedadi s27
Done:  Sedadi s32
Done:  Sedadi s33
Done:  Sedadi s34
Done:  Sedadi s37
Done:  Sedadi s38
Done:  Sedadi s39
Done:  Sidorejo Sidorejo1


In [14]:
globals()[n+model_gpm+"_gage"+str(o)]

,date,Sidorejo1,Klasifikasi
0,2000-06-18,161.923248,Sangat_Lebat
1,2000-06-26,153.513443,Sangat_Lebat
2,2000-10-22,100.365997,Sangat_Lebat
3,2000-11-28,102.147728,Sangat_Lebat
4,2000-11-29,112.120872,Sangat_Lebat
5,2001-02-08,109.614235,Sangat_Lebat
6,2001-06-20,120.996353,Sangat_Lebat
7,2001-12-23,125.413376,Sangat_Lebat
8,2002-04-29,124.398453,Sangat_Lebat
9,2002-05-09,124.227921,Sangat_Lebat


## Kebutuhan Air Irigasi

In [15]:
# read gpm data
obs = pd.read_excel("D:/Happy/RTOW/Pemali Juana/koreksi gpm/CH koreksi/Dipakai/utk koreksi fix/CH_obsQC.xlsx")
obs=obs.iloc[:,1:]
obs['date'] = pd.to_datetime(obs['date'], errors='coerce')
path_gpm =r'D:\Happy\RTOW\Pemali Juana\koreksi gpm\gpm result model\kebutuhan'
idxs_gpm = glob.glob(path_gpm + "/*.xlsx")
model_gpm=idxs_gpm[0][95:-5]

In [16]:
file_gpm = glob.glob(path_gpm + "/*.xlsx")[0]
print (idxs_gpm[0][95:-5])
init=pd.read_excel(idxs_gpm[0])
init

Klambu


,date,S1,S3,S4,S5,S7,S8,S9,S10,S11,...,S27,S30,S31,S34,S35,S36,S37,S39,S42,S43
0,01-06-2000,33.339211,44.367939,51.323380,50.336861,61.837837,48.734493,53.599819,55.763283,42.282822,...,80.161644,121.500938,113.765694,111.117805,113.947365,84.778961,91.619217,99.599800,100.549255,97.965630
1,02-06-2000,0.558634,0.415153,1.185604,1.217936,1.236601,1.456828,1.068540,1.907331,2.117617,...,5.029109,5.063179,3.011266,4.818025,6.897475,4.158597,6.573643,5.480483,8.980307,5.932275
2,03-06-2000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,04-06-2000,133.727600,105.913010,116.599815,112.556519,127.136330,133.183151,190.032578,166.729843,164.286682,...,118.645920,110.066429,111.495506,54.341057,57.490631,52.680416,67.948868,126.720573,52.319878,45.960976
4,05-06-2000,118.712959,141.752151,146.594177,102.808388,118.870483,124.546974,116.971649,85.577034,47.434395,...,76.535538,53.914875,50.892738,37.602921,34.049530,49.963146,51.478653,62.001038,24.314198,22.584122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7910,27-01-2022,0.000000,0.000000,0.000000,0.000000,3.711093,0.304804,0.304804,0.229254,0.264263,...,1.889881,1.955467,1.710598,0.591440,0.516702,2.685043,3.774378,1.703480,1.585881,2.559180
7911,28-01-2022,0.640701,0.622946,0.660665,0.553924,1.835193,2.484218,1.175392,1.710795,4.150690,...,1.585531,2.262223,1.793805,0.683707,1.054270,1.692562,1.664484,2.918572,0.672581,0.665688
7912,29-01-2022,2.204414,2.104518,5.420108,6.249417,4.422561,3.790541,3.555101,4.722488,3.845852,...,0.400052,1.314832,1.340777,4.162080,0.501605,0.766341,1.323681,0.721485,1.973151,0.571273
7913,30-01-2022,2.805296,1.537529,1.147703,0.110668,0.508876,1.026618,0.937040,0.010178,0.042835,...,0.011329,0.440732,0.355311,0.891032,0.457274,0.181320,0.070792,0.110442,0.556530,0.200908


In [20]:
#loop read excel gpm per model
path_gpm =r'D:\Happy\RTOW\Pemali Juana\koreksi gpm\gpm result model\kebutuhan'
idxs_gpm = glob.glob(path_gpm + "/*.xlsx")

for file in idxs_gpm:
    #get gpm data
    model_gpm=file[95:-5]
    file_gpm = glob.glob(path_gpm + "/*"+model_gpm+".xlsx")[0]
    globals()["gpm_"+model_gpm]=pd.read_excel(file_gpm)
    globals()["gpm_"+model_gpm]['date'] = pd.to_datetime(globals()["gpm_"+model_gpm]['date'], format="%d-%m-%Y")
    name_tmp=globals()["gpm_"+model_gpm].columns[1:].values
    
    #Klasifikasi hujan
    # loop berdasarkan urutan di excel stasiun gpm per gage tiap model
    for i in range (len(name_tmp)):
        gage=name_tmp[i]

        #create dataframe with obs data
        gpm_tmp={"date":globals()["gpm_"+model_gpm].loc[:,"date"],gage:globals()["gpm_"+model_gpm].loc[:,gage]}
        globals()[model_gpm+"_gage"+str(i)]=pd.DataFrame(gpm_tmp)

        #klasifikasi obs
        conditions = [
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] >= 0.) &\
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage]< 20),
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] >= 20) & \
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] < 50),
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] >= 50) & \
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] < 100),
            (globals()[model_gpm+"_gage"+str(i)].loc[:,gage] >= 100)
            ]

        # create a list of the values we want to assign for each condition
        values = ['Ringan','Sedang','Lebat','Sangat_Lebat']

        # create a new column and use np.select to assign values to it using our lists as arguments
        globals()[model_gpm+"_gage"+str(i)]['Klasifikasi'] = np.select(conditions, values)

        print ('Done:  '+model_gpm+' '+gage)

        #Split gpm table based on rainfall classification for each ch area
    for k in range (len(name_tmp)) :
        for j in range (len(values)) :
            globals()[values[j]+model_gpm+"_gage"+str(k)] = \
            globals()[model_gpm+"_gage"+str(k)][globals()[model_gpm+"_gage"+str(k)]['Klasifikasi']==values[j]].reset_index(drop=True)
            #print (values[j]+model_gpm+"_gage"+str(k))
                
        
        #save into excel per model dan klasifikasi
    for n in values:
        with pd.ExcelWriter('D:/Happy/RTOW/Pemali Juana/koreksi gpm/klasifikasi ch/kebutuhan/qc_'\
                            +n+'_'+model_gpm+'.xlsx') as writer: 
            for o in range (len(name_tmp)):
                gage=name_tmp[o]
                globals()[n+model_gpm+"_gage"+str(o)].to_excel(writer, sheet_name=gage)
    
    

Done:  Klambu S1
Done:  Klambu S3
Done:  Klambu S4
Done:  Klambu S5
Done:  Klambu S7
Done:  Klambu S8
Done:  Klambu S9
Done:  Klambu S10
Done:  Klambu S11
Done:  Klambu S12
Done:  Klambu S14
Done:  Klambu S15
Done:  Klambu S16
Done:  Klambu S17
Done:  Klambu S18
Done:  Klambu S19
Done:  Klambu S20
Done:  Klambu S21
Done:  Klambu S23
Done:  Klambu S24
Done:  Klambu S25
Done:  Klambu S26
Done:  Klambu S27
Done:  Klambu S30
Done:  Klambu S31
Done:  Klambu S34
Done:  Klambu S35
Done:  Klambu S36
Done:  Klambu S37
Done:  Klambu S39
Done:  Klambu S42
Done:  Klambu S43
Done:  Sedadi s6
Done:  Sedadi s7
Done:  Sedadi s8
Done:  Sedadi s12
Done:  Sedadi s13
Done:  Sedadi s14
Done:  Sedadi s15
Done:  Sedadi s19
Done:  Sedadi s20
Done:  Sedadi s21
Done:  Sedadi s22
Done:  Sedadi s25
Done:  Sedadi s26
Done:  Sedadi s27
Done:  Sedadi s32
Done:  Sedadi s33
Done:  Sedadi s34
Done:  Sedadi s37
Done:  Sedadi s38
Done:  Sedadi s39
Done:  Sidorejo Sidorejo1


In [19]:
globals()[n+model_gpm+"_gage"+str(o)]

,date,Sidorejo1,Klasifikasi
0,2000-06-18,161.923248,Sangat_Lebat
1,2000-06-26,153.513443,Sangat_Lebat
2,2000-10-22,100.365997,Sangat_Lebat
3,2000-11-28,102.147728,Sangat_Lebat
4,2000-11-29,112.120872,Sangat_Lebat
5,2001-02-08,109.614235,Sangat_Lebat
6,2001-06-20,120.996353,Sangat_Lebat
7,2001-12-23,125.413376,Sangat_Lebat
8,2002-04-29,124.398453,Sangat_Lebat
9,2002-05-09,124.227921,Sangat_Lebat
